# Step-by-step tutorial for creating simulations using `simsz`

Eve Vavagiakis et al., Last edited date

This tutorial will provide instructions on how to create galaxy cluster simulations

## Imports

In [1]:
from simsz import make_sz_cluster, dm_halo_dist

import time
import h5py

## creating simulations by hand

Create a new flat redshift and virial mass distribution using the function `flastdist_halo` from `dm_halo_dist` which uses random uniform generation. 

In [2]:
#Generate a new flat z, Mvir distribution and save to file: 
nsources=10 #Number of halos to generate
zdist,mdist=dm_halo_dist.flatdist_halo(0.01,1.1,1e14,2e15,nsources) #Generate a flat z, Mvir distribution for sims

Save our simulated data to a h5 file titled `massdist.h5`. You can find this file in the outfiles folder.

In [3]:
sourceid=int(time.time()) #Create an initial ctime for the halo ID list to save catalog
idlist=[sourceid+x for x in range(len(zdist))] #Create the halo ID list for catalog

#Save this array to a h5 file
data = h5py.File('massdist.h5', 'w')
data.create_dataset('Redshift', data=zdist)
data.create_dataset('Mass', data=mdist)
data.create_dataset('id', data=idlist)
data.close()

We can simulate submaps using our mass and redshift distribution using the `simulate_submap` function from `make_sz_cluster`. This function returns an array of dicts with the following attributes (each dict contains the full information of each sim/cluster):
- M200
- R200
- redshift_z
- y_central
- ID
- cmb_map
- noise_map
- final_map

In [4]:
#This simulates submaps for a distribution, and saves the data to a h5 file, and params to a yaml file
clusters = make_sz_cluster.simulate_T_submaps(mdist, zdist)

In [5]:
len(clusters)

10

## creating simulations automatically

Alternately, we can do everything from soup to nuts with the `simulate_clusters` class

In [6]:
c10 = make_sz_cluster.simulate_clusters(num_halos = 10)

making 10 clusters uniformly sampled from 0.1<z<1.1, 1e13<M200<1e14


In [7]:
_ = c10.get_T_maps()

This class instance populates an attribute named `clusters`, which is a dictionary. This dictionary is indexed by a run ID, and each of its values is itself a dictionary, which in turn contains two more dictionaries: `params` and `maps`

In [8]:
k10 = list(c10.clusters.keys())
k10

['8335864269041',
 '3612768891338',
 '6468438433084',
 '2779598627328',
 '6793228762617',
 '3588610311413',
 '5586119034728',
 '9415995993030',
 '6806946162090',
 '7369345445283']

In [9]:
c10.clusters[k10[0]]['params'], c10.clusters[k10[0]]['maps'].keys()

({'M200': 83358277836246.84,
  'redshift_z': 0.6487973027165306,
  'R200': 0.7493794632566557,
  'image_size_pixels': 41,
  'dT_central': -16.826445767117658},
 dict_keys(['CMB_map', 'conv_map', 'noise_map', 'final_map']))

this class instance has a `save_map` method that can save individual maps as specified, or all clusters into a single nested file with `nest_h5 = True`, or all clusters individually

In [ ]:
# c10.save_map()